# **AML Challenge: Model Stitching**

In [ ]:
# @title
from PIL import Image
import os
from urllib.request import urlopen
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import transforms

from sentence_transformers import SentenceTransformer, util
from diffusers import AutoencoderKL

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## **Downloading the Text Encoder**

In [ ]:
print("🫁 Downloading the model...")
text_encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

### **Demo for the Text Encoder:**

In [ ]:
text = "A cat is hiding under the table"

# We obtain the "embedding vector" using the encode() function:
emb = text_encoder.encode(text, convert_to_tensor=True, show_progress_bar=False)

# This is only for clarity:
preview = emb.tolist()[:3] + ["..."] + emb.tolist()[-3:]
print(f"The embedding looks like this: {preview}")

# This is the shape of our embedding:
print(f"The shape of the embedding is: {emb.shape}")

## **Downloading the VAE**

In [ ]:
print("🫁 Downloading VAE from Hugging Face...")
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to(device)

### **Demo for the VAE:**

In [ ]:
IMG_URL = "https://images.unsplash.com/photo-1574144611937-0df059b5ef3e?ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&q=80&w=764"
img = Image.open(urlopen(IMG_URL)).convert("RGB")

preprocess = transforms.Compose([
    transforms.Resize((256, 256)), # --> keep this size fixed
    # The VAE works also for 512x512, but it will require more compute
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
img_tensor = preprocess(img).unsqueeze(0).to(device)


# Encode --> latent
with torch.no_grad():
    latents = vae.encode(img_tensor).latent_dist.sample() * 0.18215
print("Latent shape:", latents.shape)

# Decode --> reconstruct
with torch.no_grad():
    recon = vae.decode(latents / 0.18215).sample

recon = (recon.clamp(-1, 1) + 1) / 2
recon_img = transforms.ToPILImage()(recon.squeeze().cpu())

In [ ]:
# Visualize input vs output:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(img.resize((256, 256)))
axes[0].set_title("Original")
axes[0].axis("off")

axes[1].imshow(recon_img)
axes[1].set_title("Reconstructed")
axes[1].axis("off")

plt.tight_layout()
plt.show()

#**Frankenstein Model:**

In [ ]:
class Translator(nn.Module):
    """
    This will be the Translator Model you have to design for the challenge.
    You have (almost) complete freedom on this. Your creativity will be rewarded.

    Some ideas might be:
    - Zero shot stitching (see https://arxiv.org/pdf/2209.15430)
    - Linear, Affine, Orthognal solutions (see https://arxiv.org/pdf/2311.00664)
    - Diffusion Priors (see https://arxiv.org/pdf/2204.06125)
    - Flow Matching (see https://arxiv.org/pdf/2412.15213)
    - CKA / Procrustes Analysis
    - Adversarial Trainings
    - AutoEncoding Solutions
    """

    def __init__(self):
        super().__init__()
        # Here is where *you* come into play:
        self.fc = nn.Linear(384, 4 * 32 * 32)
        # This is the most trivial thing you can do (spoiler: it doesn't work)

    def forward(self, x):
        x = self.fc(x)
        return x.view(1, 4, 32, 32)

translator = Translator().to(device)
translator.eval()

In [ ]:
# Part 1: encoding the text prompt
text = "Frankestein's Monster writing code on Google Colab"
print(f"Prompt: {text}")
emb = text_encoder.encode(text, convert_to_tensor=True).to(device)
print("Text embedding shape:", emb.shape)

# Part 2: translating the embedding
with torch.no_grad():
    latent = translator(emb)
print(f"Translated latent shape: {latent.shape}\n\n")

# Part 3: feed the VAE with the translation
with torch.no_grad():
    recon = vae.decode(latent / 0.18215).sample

# Part 4: visualizing the output
recon = (recon.clamp(-1, 1) + 1) / 2
recon_img = transforms.ToPILImage()(recon.squeeze().cpu())

plt.imshow(recon_img)
plt.axis("off")
plt.show()

## Baseline

In [2]:
#!mkdir data
#!gdown 1CVAQDuPOiwm8h9LJ8a_oOs6zOWS6EgkB
#!gdown 1ykZ9fjTxUwdiEwqagoYZiMcD5aG-7rHe
#!unzip -o test.zip -d data
#!unzip -o train.zip -d data
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/Mamiglia/challenge.git

Mounted at /content/drive
Cloning into 'challenge'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 98 (delta 39), reused 72 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 21.03 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from pathlib import Path
from tqdm import tqdm

from challenge.src.common import load_data, prepare_train_data, generate_submission

In [34]:
class TransformerTranslator(nn.Module):
    """
    Transformer-style translator from text embedding -> image embedding
    """
    def __init__(self, text_dim=1024, img_dim=1536, n_heads=8, n_layers=2, dim_feedforward=2048, dropout=0.2):
        super().__init__()
        self.input_ln = nn.LayerNorm(text_dim)
        self.proj_in = nn.Linear(text_dim, img_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=img_dim,
            nhead=n_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='gelu',
            batch_first=True  # for (B, Seq, Dim)
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.output_ln = nn.LayerNorm(img_dim)

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)  # (B, 1, text_dim)
        x = self.input_ln(x)
        x = self.proj_in(x)  # project to model dim
        out = self.encoder(x)  # Transformer encoder
        out = out.squeeze(1)   # remove sequence dim
        return self.output_ln(out)

In [52]:
class ResidualMLPTranslator(nn.Module):
    def __init__(self, text_dim=1024, img_dim=1536, hidden_dim=2048, num_layers=3, dropout=0.2):
        super().__init__()
        assert num_layers >= 2
        self.input_ln = nn.LayerNorm(text_dim)
        self.dropout = nn.Dropout(dropout)

        # first layer: text_dim -> hidden_dim (no residual yet)
        self.first_layer = nn.Sequential(
            nn.Linear(text_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout)
        )

        # hidden residual blocks (hidden_dim -> hidden_dim)
        self.blocks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.GELU(),
                nn.LayerNorm(hidden_dim),
                nn.Dropout(dropout)
            )
            for _ in range(num_layers - 2)
        ])

        # final projection to image space
        self.final_proj = nn.Linear(hidden_dim, img_dim)
        self.output_ln = nn.LayerNorm(img_dim)

        # input residual to output
        if text_dim != img_dim:
            self.res_proj = nn.Linear(text_dim, img_dim)
        else:
            self.res_proj = nn.Identity()

    def forward(self, x):
        x_in = self.input_ln(x)
        out = self.first_layer(x_in)
        for block in self.blocks:
            out = out + block(out)  # residual only between same-dim layers
        out = self.final_proj(out)
        out = out + self.res_proj(x_in)
        return self.output_ln(out)


In [ ]:
class LatentSpaceTranslator(nn.Module):
    """
    MLP translator from text embedding -> image embedding
    Input: text_emb (batch, text_dim) or (batch, 1, text_dim)
    Output: (batch, img_dim)
    Regularization: dropout, LayerNorm, GELU, residual (optional projector)
    """
    def __init__(self,
                 text_dim=1024,
                 img_dim=1536,
                 hidden_dim=2048,
                 num_layers=3,
                 dropout=0.2,
                 use_residual=True):
        super().__init__()
        assert num_layers >= 2, "num_layers should be >= 2 (including final proj)"
        self.use_residual = use_residual
        self.input_ln = nn.LayerNorm(text_dim)
        layers = []
        in_dim = text_dim
        for i in range(num_layers - 1):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.LayerNorm(hidden_dim))
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dim
        # final projection to image space
        layers.append(nn.Linear(in_dim, img_dim))
        self.net = nn.Sequential(*layers)

        # if using residual, project input to img_dim to add it at the end
        if self.use_residual:
            if text_dim != img_dim:
                self.res_proj = nn.Linear(text_dim, img_dim)
            else:
                self.res_proj = nn.Identity()

        # final layer norm in image space
        self.output_ln = nn.LayerNorm(img_dim)

    def forward(self, text_emb):
        if text_emb.dim() == 3:
            x = text_emb.squeeze(1)
        else:
            x = text_emb
        x = self.input_ln(x)
        out = self.net(x)  # (B, img_dim)
        if self.use_residual:
            res = self.res_proj(x)
            out = out + res
        return self.output_ln(out)


In [36]:
# ---------- Training loop with Procrustes + InfoNCE ----------
def training(model, train_loader, val_loader, device, epochs, lr, MODEL_PATH,
             use_procrustes_init=True, procrustes_subset=10000, temperature=0.07,
             w_nce = 0.2, w_mse = 0.2, w_cos = 0.2):
    """Train LatentSpaceTranslator with optional Procrustes init + InfoNCE loss."""
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-3)
    best_val_loss = float('inf')

    # --- Optional: Procrustes initialization ---
    if use_procrustes_init:
        print("Computing Procrustes initialization...")
        text_list, img_list = [], []
        for i, (X, y) in enumerate(train_loader):
            text_list.append(X.cpu())
            img_list.append(y.cpu())
            if sum(t.shape[0] for t in text_list) >= procrustes_subset:
                break
        text_sample = torch.cat(text_list, dim=0)[:procrustes_subset]
        img_sample = torch.cat(img_list, dim=0)[:procrustes_subset]
        model = apply_procrustes_init_to_final(model, text_sample, img_sample)

    # --- Training ---
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            pred = model(X_batch)
            # Contrastive InfoNCE loss (can mix with cosine or MSE)
            loss = w_nce * info_nce_loss(pred, y_batch, temperature=temperature)
            # Optional mixed objective:
            loss += w_cos * (1 - F.cosine_similarity(pred, y_batch).mean())
            loss += w_mse * F.mse_loss(pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # --- Validation ---
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                pred = model(X_batch)
                loss = w_nce * info_nce_loss(pred, y_batch, temperature=temperature)
                # Optional mixed objective:
                loss += w_cos * (1 - F.cosine_similarity(pred, y_batch).mean())
                loss += w_mse * F.mse_loss(pred, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}")

        # --- Save best model ---
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            Path(MODEL_PATH).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"  ✓ Saved best model (val_loss={val_loss:.6f})")

    return model

In [37]:
# ====== Procrustes initialization ======
def procrustes_init(text_embs, img_embs):
    """
    text_embs: (N, d_text)
    img_embs:  (N, d_img)
    returns: weight matrix (d_img, d_text)
    """
    # Center both
    X = text_embs - text_embs.mean(0, keepdim=True)
    Y = img_embs - img_embs.mean(0, keepdim=True)

    # Compute SVD of cross-covariance
    U, _, Vt = torch.linalg.svd(X.T @ Y, full_matrices=False)
    W = U @ Vt  # orthogonal map d_text→d_img
    return W.T   # shape (d_img, d_text) for nn.Linear weight

# ====== InfoNCE (CLIP-style) loss ======
def info_nce_loss(pred_img_emb, true_img_emb, temperature=0.07):
    """
    pred_img_emb: (B, D)
    true_img_emb: (B, D)
    """
    zt = F.normalize(pred_img_emb, dim=1)
    zi = F.normalize(true_img_emb, dim=1)
    logits = zt @ zi.t() / temperature
    labels = torch.arange(len(zt), device=zt.device)
    loss_i2t = F.cross_entropy(logits, labels)
    loss_t2i = F.cross_entropy(logits.t(), labels)
    return 0.5 * (loss_i2t + loss_t2i)

def apply_procrustes_init_to_final(model, text_embs, img_embs):
    """Initialize the final linear layer of model with a Procrustes alignment."""
    W = procrustes_init(text_embs, img_embs)  # (img_dim, text_dim)
    with torch.no_grad():
        # find the last Linear layer (hidden_dim -> img_dim)
        final_linear = None
        for m in reversed(list(model.modules())):
            if isinstance(m, nn.Linear):
                final_linear = m
                break
        if final_linear is not None:
            # Resize if hidden_dim != text_dim
            rows, cols = final_linear.weight.shape
            # compress or pad W.T accordingly
            if cols != W.shape[1]:
                # project W to match
                U, S, Vt = torch.linalg.svd(W, full_matrices=False)
                W_approx = (U[:, :cols] @ Vt[:cols, :])  # (img_dim, cols)
            else:
                W_approx = W

            # If shape still doesn't match, slice
            W_approx = W_approx[:rows, :cols]
            final_linear.weight.copy_(W_approx)
            print(f"✓ Applied Procrustes init to final layer: {rows}×{cols}")
    return model


In [38]:
# 1. Learning with a basic simple transformer
# 2. Learning with a basic deep neural network
# 3. TODO: Creat a hyperparameter optimization for the transformer aswell as
# the lost function nce loss etc. and perform it on the first layer of the transformer
# Configuration
EPOCHS = 30
BATCH_SIZE = 256
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load data
train_data = load_data("drive/MyDrive/data/train/train.npz")
X, y, label = prepare_train_data(train_data)
DATASET_SIZE = len(X)
# Split train/val
# This is done only to measure generalization capabilities, you don't have to
# use a validation set (though we encourage this)
n_train = int(0.9 * len(X))
TRAIN_SPLIT = torch.zeros(len(X), dtype=torch.bool)
TRAIN_SPLIT[:n_train] = 1
X_train, X_val = X[TRAIN_SPLIT], X[~TRAIN_SPLIT]
y_train, y_val = y[TRAIN_SPLIT], y[~TRAIN_SPLIT]


train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
y_train.shape, X_train.shape, train_loader.batch_size, val_loader.batch_size

(125000,)
Train data: 125000 captions, 125000 images


(torch.Size([112500, 1536]), torch.Size([112500, 1024]), 256, 256)

## Hyperparameter Optimization

In [39]:
!pip install optuna

In [50]:
import optuna

def objective_extended(arch, trial, train_dataloader, val_dataloader, device, MODEL_PATH_BASE):
    # --- Sample hyperparameters ---
    hidden_dim = trial.suggest_categorical("hidden_dim", [1024, 2048, 4096])
    num_layers = trial.suggest_int("num_layers", 2, 6)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])

    # --- New hyperparameters ---
    temperature = trial.suggest_float("temperature", 0.01, 0.2)
    w_infonce = trial.suggest_float("w_infonce", 0.6, 0.8)
    w_cos = trial.suggest_float("w_cos", 0.4, 1.0)
    w_mse = trial.suggest_float("w_mse", 1.0 - w_cos, 1.0)
    procrustes_subset = 10000

    # --- Create model ---

    if arch == 'transformer':
      model = TransformerTranslator().to(DEVICE)

    elif arch == 'nn':
        model = LatentSpaceTranslator(
        text_dim=1024,
        img_dim=1536,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout).to(device)

    else:
        model = ResidualMLPTranslator(
        text_dim=1024,
        img_dim=1536,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout).to(DEVICE)


    # --- Apply Procrustes initialization ---
    if procrustes_subset > 0:
        # Get subset from train_loader
        text_list, img_list = [], []
        for i, (X, y) in enumerate(train_loader):
            text_list.append(X.cpu())
            img_list.append(y.cpu())
            if sum(t.shape[0] for t in text_list) >= procrustes_subset:
                break
        text_sample = torch.cat(text_list, dim=0)[:procrustes_subset]
        img_sample = torch.cat(img_list, dim=0)[:procrustes_subset]
        model = apply_procrustes_init_to_final(model, text_sample, img_sample)

    # --- Training loop (short run) ---
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.train()
    for epoch in range(5):  # short training
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            pred = model(X_batch)
            pred = F.normalize(pred, dim=-1)
            y_batch = F.normalize(y_batch, dim=-1)

            # Weighted combination of losses
            loss = 0
            if w_infonce > 0:
                loss += w_infonce * info_nce_loss(pred, y_batch, temperature=temperature)
            if w_mse > 0:
                loss += w_mse * F.mse_loss(pred, y_batch)
            if w_cos > 0:
                loss += w_cos * (1 - F.cosine_similarity(pred, y_batch).mean())

            loss.backward()
            optimizer.step()

    # --- Evaluate on validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            pred = model(X_batch)
            pred = F.normalize(pred, dim=-1)
            y_batch = F.normalize(y_batch, dim=-1)
            # Use combined loss for evaluation
            loss = 0
            if w_infonce > 0:
                loss += w_infonce * info_nce_loss(pred, y_batch, temperature=temperature)
            if w_mse > 0:
                loss += w_mse * F.mse_loss(pred, y_batch)
            if w_cos > 0:
                loss += w_cos * (1 - F.cosine_similarity(pred, y_batch).mean())
            val_loss += loss.item()
    val_loss /= len(val_loader)

    return val_loss


def run_optuna_extended(arch, train_dataloader, val_dataloader, device, MODEL_PATH_BASE, n_trials=10):
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective_extended(arch, trial, train_dataloader, val_dataloader, device, MODEL_PATH_BASE),
                   n_trials=n_trials)

    print("Best trial:")
    trial = study.best_trial
    print(f"Val loss: {trial.value}")
    print("Best hyperparameters:")
    for key, value in trial.params.items():
        print(f"  {key}: {value}")

    return trial.params

In [53]:
arch = 'res'
best_params = run_optuna_extended(
    arch = 'res',
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    device=DEVICE,
    MODEL_PATH_BASE="models/translator_optuna"
)

if arch == 'transformer':
    model = TransformerTranslator().to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data//models/transformer.pth"

elif arch == 'nn':
    model = LatentSpaceTranslator(
    text_dim=1024,
    img_dim=1536,
    hidden_dim=best_params["hidden_dim"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"]).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data/models/latent_space.pth"

else:
    model = ResidualMLPTranslator(
    text_dim=1024,
    img_dim=1536,
    hidden_dim=best_params["hidden_dim"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"]).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data/models/residual.pth"

[I 2025-10-31 11:33:44,983] A new study created in memory with name: no-name-91d0b3c8-efa9-42b8-88e5-131817155c7b
/tmp/ipython-input-916805747.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)
/tmp/ipython-input-916805747.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)


✓ Applied Procrustes init to final layer: 1536×1024


[I 2025-10-31 11:34:08,362] Trial 0 finished with value: 3.3108338871780707 and parameters: {'hidden_dim': 2048, 'num_layers': 3, 'dropout': 0.36071903056866395, 'weight_decay': 0.00020792509380118625, 'lr': 1.2651928325672493e-05, 'batch_size': 64, 'temperature': 0.19896949396211575, 'w_infonce': 0.7007854220900475, 'w_cos': 0.6747202940848545, 'w_mse': 0.5432330993195925}. Best is trial 0 with value: 3.3108338871780707.


✓ Applied Procrustes init to final layer: 1536×1024


[I 2025-10-31 11:34:28,433] Trial 1 finished with value: 2.5476222086925895 and parameters: {'hidden_dim': 1024, 'num_layers': 5, 'dropout': 0.24034302193434762, 'weight_decay': 4.217153743745833e-05, 'lr': 1.347217160770757e-05, 'batch_size': 64, 'temperature': 0.11376524249411407, 'w_infonce': 0.6119678078943391, 'w_cos': 0.7392104202237512, 'w_mse': 0.45028797776702034}. Best is trial 1 with value: 2.5476222086925895.


✓ Applied Procrustes init to final layer: 1536×1024


[W 2025-10-31 11:34:30,658] Trial 2 failed with parameters: {'hidden_dim': 2048, 'num_layers': 6, 'dropout': 0.24365242711057036, 'weight_decay': 0.00013847890031289011, 'lr': 0.0008006774692787673, 'batch_size': 256, 'temperature': 0.17450242429350432, 'w_infonce': 0.7763051268812414, 'w_cos': 0.56025858887087, 'w_mse': 0.9779758918753335} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-916805747.py", line 102, in <lambda>
    study.optimize(lambda trial: objective_extended(arch, trial, train_dataloader, val_dataloader, device, MODEL_PATH_BASE),
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-916805747.py", line 68, in objective_extended
    loss += w_i

KeyboardInterrupt: 

In [44]:
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train
print("\n3. Training...")
model = training(model,
                 train_loader,
                 val_loader,
                 DEVICE,
                 EPOCHS,
                 best_params["lr"],
                 MODEL_PATH,
                 True,
                 10000,
                 best_params["temperature"],
                 best_params["w_infonce"],
                 best_params["w_mse"],
                 best_params["w_cos"])

# Load best model for evaluation
model.load_state_dict(torch.load(MODEL_PATH))

   Parameters: 6,830,080

3. Training...
Computing Procrustes initialization...
✓ Applied Procrustes init to final layer: 1536×1024


Epoch 1/30: 100%|██████████| 440/440 [00:04<00:00, 101.15it/s]


Epoch 1: Train Loss = 1.739060, Val Loss = 2.079712
  ✓ Saved best model (val_loss=2.079712)


Epoch 2/30: 100%|██████████| 440/440 [00:04<00:00, 93.82it/s] 


Epoch 2: Train Loss = 1.404553, Val Loss = 2.005234
  ✓ Saved best model (val_loss=2.005234)


...

Epoch 4: Train Loss = 1.202754, Val Loss = 1.926914
  ✓ Saved best model (val_loss=1.926914)


Epoch 5/30: 100%|██████████| 440/440 [00:04<00:00, 98.47it/s] 


Epoch 5: Train Loss = 1.126508, Val Loss = 1.899866
  ✓ Saved best model (val_loss=1.899866)


Epoch 6/30:  80%|███████▉  | 351/440 [00:03<00:00, 111.00it/s]


KeyboardInterrupt: 

In [43]:
from challenge.src.eval import visualize_retrieval
import numpy as np
import torch

val_caption_text = train_data['captions/text'][~TRAIN_SPLIT]
val_text_embd = X_val
img_VAL_SPLIT = label[~TRAIN_SPLIT].sum(dim=0) > 0
val_img_file = train_data['images/names'][img_VAL_SPLIT]
val_img_embd = torch.from_numpy(train_data['images/embeddings'][img_VAL_SPLIT])
val_label = np.nonzero(train_data['captions/label'][~TRAIN_SPLIT][:,img_VAL_SPLIT])[1]

# Sample and visualize
for i in range(5):
    idx = np.random.randint(0, 100)
    caption_embd = val_text_embd[idx]
    caption_text = val_caption_text[idx]
    gt_index = val_label[idx]

    with torch.no_grad():
        pred_embds = model(caption_embd.to(DEVICE)).cpu()

        visualize_retrieval(
            pred_embds,
            gt_index,
            val_img_file,
            caption_text, val_img_embd, k=5,
            dataset_path = 'drive/MyDrive/data/train')

KeyboardInterrupt: 

In [32]:
test_data = load_data("drive/MyDrive/data/test/test.clean.npz")

test_embds = test_data['captions/embeddings']
test_embds = torch.from_numpy(test_embds).float()

with torch.no_grad():
    pred_embds = model(test_embds.to(DEVICE)).cpu()

submission = generate_submission(test_data['captions/ids'], pred_embds, 'submission.csv')
print(f"Model saved to: {MODEL_PATH}")

Generating submission file...
✓ Saved submission to submission.csv
Model saved to: drive/MyDrive/data/models/latent_space.pth
